#***Trabalhando com o Google Earth Engine*** 



1.   Importar Coleções
2.   Selecionar uma área de estudo
3.   Calculo dos índices 
4.   Construíndo Gráficos

In [ ]:
#Instalando as bibliotecas
!pip install earthengine-api
!pip install geopandas
!pip install geemap

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 22.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.6/99.6 KB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 24.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 58.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 KB 7.1 M

In [ ]:
##Montando o acesso ao google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Importando as bibliotecas
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=YozoKh--wUx6JU57R8QkLjYxirFS1lxH1Tpoq3ZqMbQ&tc=3KnZvpLk0Ndw7P_ygl0TumbdP6wRmzxs9M2i0PQKCRI&cc=GGU5nZ0YvuFUya3bMiMbhK7ODFnksFr1Icy7Sy7434k

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AWtgzh4ilf8fpf6n3im6Gf7Rgcezd32UhfRHnN_sa2VeWgdoUiWsmZtSOwg

Successfully saved authorization token.


In [ ]:
##Importe as bibliotecas e pacotes que utilizaremos
import geopandas as gpd 
import json
import pandas as pd
import geemap
import geemap.colormaps as cm
import altair as alt ##biblioteca de visualização(gráficos)
import numpy as np ## pacote de processamento de matriz (regressão linear)

In [ ]:
##Convertendo arquivo shp to json to feature
##Lendo o arquivo shp
gdf = gpd.read_file('/content/drive/MyDrive/BASES/RS_Municipios_2021.shp') 
gdf.head(5)

,CD_MUN,NM_MUN,SIGLA,AREA_KM2,geometry
0,4300001,Lagoa Mirim,RS,2872.364,"POLYGON ((-52.62347 -32.14606, -52.62286 -32.1..."
1,4300002,Lagoa dos Patos,RS,10196.942,"POLYGON ((-51.29028 -30.00222, -51.28939 -30.0..."
2,4300034,Aceguá,RS,1551.339,"POLYGON ((-54.10842 -31.43275, -54.10739 -31.4..."
3,4300059,Água Santa,RS,291.503,"POLYGON ((-52.04086 -28.12188, -52.04031 -28.1..."
4,4300109,Agudo,RS,534.624,"POLYGON ((-53.25392 -29.44907, -53.25371 -29.4..."


In [ ]:
 ##Filtrando as colunas que temos mais interesse
 roi = gdf.loc[:,['NM_MUN','geometry']]
 roi = roi[roi.NM_MUN=='Agudo']
 roi

,NM_MUN,geometry
4,Agudo,"POLYGON ((-53.25392 -29.44907, -53.25371 -29.4..."


# ***Preparando nossa featurecollection.***

O GEE não permite a utilização direta de um arquivo .shp. Sendo assim nos iremos converter essa base para json. 

* Veja mais sobre FeatureCollection em [featurecollection](https://developers.google.com/earth-engine/guides/feature_collection_filtering)

In [ ]:
##convertendo de shp para to json
roi = roi.to_json()
##Carregando o arquivo json
roi = json.loads(roi)
##selecionando as features
roi = roi['features']
##Verificando o processo
roi

[{'id': '4',
  'type': 'Feature',
  'properties': {'NM_MUN': 'Agudo'},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-53.2539234459999, -29.449070865999936],
     [-53.25370783599992, -29.450221683999942],
     [-53.25063089999997, -29.45036047199997],
     [-53.24909004399993, -29.45187369099994],
     [-53.24895531999998, -29.455890132999997],
     [-53.246519728999985, -29.455984627],
     [-53.24561416099993, -29.457859543999966],
     [-53.24338835599997, -29.457460800000014],
     [-53.24227495599995, -29.458022811999975],
     [-53.24024783899992, -29.46112000599991],
     [-53.23530711899995, -29.465320003999963],
     [-53.23335812699998, -29.46483607699998],
     [-53.231429406999915, -29.465494335999907],
     [-53.23015371799997, -29.464727370999952],
     [-53.227383634999924, -29.466212464999998],
     [-53.22689560599996, -29.465039569999938],
     [-53.22449499699992, -29.46497598900001],
     [-53.225899816999984, -29.466789326999944],
     [-53.22507492899994

In [ ]:
##Definindo filtros na feature
region = ee.FeatureCollection(roi)

In [ ]:
# Defina um método para exibir blocos de imagens do Earth Engine em um folium map
Map = geemap.Map()
##Centralizando a imagem
Map = geemap.Map(location=[-29.5543, -53.1683], zoom_start= 10)
#Para ver uma visualização de satélite do Google como um mapa base
Map.add_basemap('HYBRID')
##Adicionando a feature  = Converte para Image (desenha o limite)
image = ee.Image().paint(region, 0, 2)
Map.addLayer(image,{'palette':'red'}, 'ROI')

display(Map)

Map(center=[-29.5543, -53.1683], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

# ***Preparando nossa coleção de imagens.***

* Pré-processamento
* Verificando propriedades 
* Índices
 

In [ ]:
##Máscara de nuvens 
def maskL8sr(image):

    qaMask = image.select('QA_PIXEL').bitwise_and(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    ###Fator de escala
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBands = image.select('ST_B.*').multiply(0.00341802).add(149.0)

    return image.addBands(opticalBands, None, True)\
    .addBands(thermalBands, None, True).updateMask(qaMask)\
    .updateMask(saturationMask)

In [ ]:
###Definição de entradas - Período de interesse
start_date = '2015-01-01'
end_date = '2021-08-28'

##Função NDVI
def Ndvi(img):  
    Ndvi_image = img.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return img.addBands(Ndvi_image).copyProperties(img, ["system:time_start"]).set('date', img.date().format('YYYY-MM-dd'))

#Importando imagem Landsat 8 collection SR e aplicando filtros
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
                                .filterBounds(region)\
                                .filterDate(start_date, end_date)\
                                .filter(ee.Filter.lt('CLOUD_COVER',10))\
                                .map(maskL8sr)\
                                .map(Ndvi)   
print('Qtds',l8.size().getInfo())                  

Qtds 172


In [ ]:
#NDVI reduce image
Ndvi = l8.select('NDVI').median().clip(region)
##Colors map
palette = cm.palettes.ndvi
##Definindo os parâmetros
vis_params = {
  'min': 0,
  'max': 1,
  'palette': palette}

##Adicionando o Layer 
Map.add_ee_layer(l8.median().clip(region), {'bands': ['SR_B4', 'SR_B3', 'SR_B2'],'min': 0.03,'max': 0.19}, 'RGB')
Map.addLayer(Ndvi, vis_params, 'NDVI')

##Adicionando a feature  = Converte para Image (desenha o limite)
image = ee.Image().paint(region, 0, 2)
Map.addLayer(image,{'palette':'red'}, 'ROI')
Map.add_colorbar(vis_params, label="NDVI", layer_name="NDVI")

##Finalizando a tarefa
display(Map)

Map(center=[-29.5543, -53.1683], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(c…

In [ ]:
###Definindo lista de datas 
def date (image):
    return image.clip(region).copyProperties(image, ["system:time_start"]).set('date', image.date().format('YYYY-MM-dd')).set({'millis': image.date().millis()})
##Aplicando a função
l8 = l8.map(date)
l8.first().propertyNames().getInfo()

['date',
 'millis',
 'DATA_SOURCE_ELEVATION',
 'WRS_TYPE',
 'system:id',
 'REFLECTANCE_ADD_BAND_1',
 'REFLECTANCE_ADD_BAND_2',
 'DATUM',
 'REFLECTANCE_ADD_BAND_3',
 'REFLECTANCE_ADD_BAND_4',
 'REFLECTANCE_ADD_BAND_5',
 'REFLECTANCE_ADD_BAND_6',
 'REFLECTANCE_ADD_BAND_7',
 'REFLECTIVE_SAMPLES',
 'system:version',
 'GROUND_CONTROL_POINTS_VERSION',
 'SUN_AZIMUTH',
 'DATA_SOURCE_TIRS_STRAY_LIGHT_CORRECTION',
 'UTM_ZONE',
 'DATE_ACQUIRED',
 'ELLIPSOID',
 'system:time_end',
 'DATA_SOURCE_PRESSURE',
 'LANDSAT_PRODUCT_ID',
 'STATION_ID',
 'TEMPERATURE_ADD_BAND_ST_B10',
 'DATA_SOURCE_REANALYSIS',
 'REFLECTANCE_MULT_BAND_7',
 'system:time_start',
 'REFLECTANCE_MULT_BAND_6',
 'L1_PROCESSING_LEVEL',
 'PROCESSING_SOFTWARE_VERSION',
 'L1_DATE_PRODUCT_GENERATED',
 'ORIENTATION',
 'REFLECTANCE_MULT_BAND_1',
 'WRS_ROW',
 'REFLECTANCE_MULT_BAND_3',
 'REFLECTANCE_MULT_BAND_2',
 'TARGET_WRS_ROW',
 'REFLECTANCE_MULT_BAND_5',
 'REFLECTANCE_MULT_BAND_4',
 'THERMAL_LINES',
 'TIRS_SSM_POSITION_STATUS',
 'GRID_

# ***Estatísticas de nossa área de estudo***

Nesta etapa vamos extrair as estatísticas por região (área de estudo) e gerar nosso dataframe para criar gráficos.

Para obter estatísticas de imagem em várias regiões armazenadas em um FeatureCollection, você pode usar image.reduceRegions() para reduzir várias regiões de uma vez. A entrada para reduceRegions() é uma Image e uma FeatureCollection. A saída é outra FeatureCollection com a saída reduceRegions() definida como propriedades em cada Feature. Neste exemplo, as médias das bandas  do Landsat 8 em cada geometria de recurso serão adicionadas como propriedades aos recursos de entrada:

[fonte](https://developers.google.com/earth-engine/guides/reducers_reduce_regions)

<center>
<image src =https://developers.google.com/earth-engine/images/Reduce_region_diagram.png>
<center> Figure 1. An illustration of an ee.Reducer applied to an image and a region.

In [ ]:
##Função para converter informações da imagem em tabela
def reduce (image):
    serie_reduce = image.reduceRegions(**{
                        'collection':region,
                        'reducer': ee.Reducer.mean().combine(**{
                        'reducer2': ee.Reducer.min(), 
                                    'sharedInputs': True}).combine(**{
                        'reducer2': ee.Reducer.max(),
                                    'sharedInputs': True}), 
                        'scale': 30
                        })
     
    serie_reduce = serie_reduce.map(lambda f: f.set({'millis': image.get('millis')}))\
                                .map(lambda f: f.set({'date': image.get('date')}))         

    return serie_reduce.copyProperties(image, ["system:time_start"])

##Aplicando a função de redução na Coleção 
data_reduce = l8.select('NDVI').map(reduce)\
                        .flatten()\
                        .sort('date',True)\
                        .select(['millis','date','min','mean','max'])

##Verificando os dados e as propriedades
print(data_reduce.size().getInfo())
data_reduce.first().propertyNames().getInfo()

172


['system:index', 'date', 'min', 'max', 'mean', 'millis']

In [ ]:
##Estabelecendo a lista dos dados
Lista_df = data_reduce.reduceColumns(ee.Reducer.toList(5), ['millis','date','min','mean','max']).values().get(0)

In [ ]:
# não se esqueça que precisamos chamar o método de retorno de chamada "getInfo" para recuperar os dados
df_ndvi = pd.DataFrame(Lista_df.getInfo(), columns=['millis','date','NDVI_min','NDVI_mean','NDVI_max'])
df_ndvi

,millis,date,NDVI_min,NDVI_mean,NDVI_max
0,1420551043572,2015-01-06,-0.381926,0.774956,0.932091
1,1421933437803,2015-01-22,-0.683296,0.816440,0.968896
2,1421933461753,2015-01-22,-0.709353,0.824041,0.966003
3,1423315834641,2015-02-07,-0.983004,0.800267,0.988838
4,1423315858587,2015-02-07,-0.936765,0.821225,0.981889
...,...,...,...,...,...
160,1620393863329,2021-05-07,-0.993718,0.683125,0.999514
161,1622381424501,2021-05-30,-0.932079,0.304642,0.932336
162,1622381448452,2021-05-30,-0.895010,0.248978,0.888885
163,1625923464253,2021-07-10,-0.976046,0.747197,0.999620


In [ ]:
##Exportar tabela para o drive
export_precipit = df_ndvi.to_csv ('/content/drive/MyDrive/GEE/tabela_NDVI_aula10.csv')

In [ ]:
##Dataframe
print(df_ndvi)

            millis        date  NDVI_min  NDVI_mean  NDVI_max
0    1420551043572  2015-01-06 -0.381926   0.774956  0.932091
1    1421933437803  2015-01-22 -0.683296   0.816440  0.968896
2    1421933461753  2015-01-22 -0.709353   0.824041  0.966003
3    1423315834641  2015-02-07 -0.983004   0.800267  0.988838
4    1423315858587  2015-02-07 -0.936765   0.821225  0.981889
..             ...         ...       ...        ...       ...
160  1620393863329  2021-05-07 -0.993718   0.683125  0.999514
161  1622381424501  2021-05-30 -0.932079   0.304642  0.932336
162  1622381448452  2021-05-30 -0.895010   0.248978  0.888885
163  1625923464253  2021-07-10 -0.976046   0.747197  0.999620
164  1625923488203  2021-07-10 -0.999142   0.649307  0.999111

[165 rows x 5 columns]


In [ ]:
#Estatística descritiva
df_ndvi.describe()

,millis,NDVI_min,NDVI_mean,NDVI_max
count,1.650000e+02,165.000000,165.000000,165.000000
mean,1.532887e+12,-0.868779,0.619186,0.958668
std,5.994646e+10,0.174214,0.189135,0.043788
min,1.420551e+12,-0.999710,0.012933,0.781140
25%,1.484919e+12,-0.984498,0.459769,0.929412
50%,1.540820e+12,-0.932755,0.690758,0.974918
75%,1.583674e+12,-0.824568,0.771121,0.994237
max,1.625923e+12,-0.086051,0.853957,0.999942


In [ ]:
# Função para adicionar variáveis de data ao DataFrame.
def add_date_info(df):
  df['Timestamp'] = pd.to_datetime(df['millis'], unit='ms')
  df['Year'] = pd.DatetimeIndex(df['Timestamp']).year
  df['Month'] = pd.DatetimeIndex(df['Timestamp']).month
  df['Day'] = pd.DatetimeIndex(df['Timestamp']).day
  df['DOY'] = pd.DatetimeIndex(df['Timestamp']).dayofyear
  return df

In [ ]:
##Adicionando as datas no Dataframe
df_ndvi = add_date_info(df_ndvi)
df_ndvi.head(5)

,millis,date,NDVI_min,NDVI_mean,NDVI_max,Timestamp,Year,Month,Day,DOY
0,1420551043572,2015-01-06,-0.381926,0.774956,0.932091,2015-01-06 13:30:43.572,2015,1,6,6
1,1421933437803,2015-01-22,-0.683296,0.816440,0.968896,2015-01-22 13:30:37.803,2015,1,22,22
2,1421933461753,2015-01-22,-0.709353,0.824041,0.966003,2015-01-22 13:31:01.753,2015,1,22,22
3,1423315834641,2015-02-07,-0.983004,0.800267,0.988838,2015-02-07 13:30:34.641,2015,2,7,38
4,1423315858587,2015-02-07,-0.936765,0.821225,0.981889,2015-02-07 13:30:58.587,2015,2,7,38


In [ ]:
##Cáculo da média dos meses
NDVI_month=df_ndvi.groupby(['Month'])['NDVI_mean'].mean()
NDVI_month

Month
1     0.803925
2     0.820725
3     0.753005
4     0.548381
5     0.509980
6     0.588934
7     0.505699
8     0.561851
9     0.549049
10    0.578547
11    0.477501
12    0.616423
Name: NDVI_mean, dtype: float64

In [ ]:
#NDVI mensal
NDVI_month.describe()

count    12.000000
mean      0.609502
std       0.117629
min       0.477501
25%       0.538780
50%       0.570199
75%       0.650568
max       0.820725
Name: NDVI_mean, dtype: float64

In [ ]:
##Type
type(NDVI_month)

pandas.core.series.Series

# ***Gerando Gráficos***

Neste exemplo vamos utilizar a biblioteca Altair [click aqui e saiba mais](https://altair-viz.github.io/).

In [ ]:
##Gráfico Série Temporal
alt.Chart(df_ndvi).mark_line().encode(
    x=alt.X('date', title='Data',axis=alt.Axis(labelAngle=-75)),
    y=alt.Y('NDVI_mean:Q', title='NDVI', scale=alt.Scale(domain=[0.5, 0.9])),
    
    tooltip=[
             alt.Tooltip('date:N', title='Data'),
             alt.Tooltip('NDVI_mean:Q')
             ]).properties(title='Série histórica NDVI por ano',width=1500, height=300).interactive()

alt.Chart(...)

In [ ]:
##Série Temporal em colunas
alt.Chart(df_ndvi).mark_bar(size=6).encode(
     
     x=alt.X('date:T',title='Data'),
     y=alt.Y('NDVI_mean:Q', title='NDVI'),
    color=alt.Color(
        'NDVI_mean:Q', scale=alt.Scale(scheme='redyellowgreen', domain=(-1, 1))),
    tooltip=[
        alt.Tooltip('date:T', title='Date'),
        alt.Tooltip('NDVI_mean:Q', title='NDVI')
    ]).properties(title='Série histórica NDVI por ano',width=1000, height=300)

alt.Chart(...)

In [ ]:
##Visualizando o resultado com o pacote ALTAIR
highlight = alt.selection(
    type='single', on='mouseover', fields=['Year'], nearest=True)

## Preparando nossa base para um gráfico por ano e por dia do ano.
base = alt.Chart(df_ndvi).encode(
    x=alt.X('DOY:Q', scale=alt.Scale(domain=[0, 364], clamp=True)),
    y=alt.Y('NDVI_mean:Q', title='NDVI', scale=alt.Scale(domain=[0.2, 0.85])),
    color=alt.Color('Year:O', scale=alt.Scale(scheme='redyellowgreen')))

## Definindo  os parâmetros e legenda
points = base.mark_circle().encode(
    opacity=alt.value(0),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('DOY:Q', title='DOY'),
        alt.Tooltip('NDVI_mean:Q', title='NDVI')
    ]).add_selection(highlight)

lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1), alt.value(3)))

(points + lines).properties(title='Série histórica NDVI por ano',width=800, height=600).interactive()

alt.LayerChart(...)

In [ ]:
##Visualização dos dados com base Altair
base = alt.Chart(df_ndvi).encode(
    x=alt.X('Month:Q', scale=alt.Scale(domain=(1,12)),axis=alt.Axis(labelAngle=-75)))
    

line = base.mark_line().encode(
    y=alt.Y('NDVI_mean:Q',aggregate='mean', scale=alt.Scale(domain=(0.20, 1))))

band = base.mark_errorband(extent='iqr').encode(
   y=alt.Y('NDVI_mean:Q',title='NDVI'))

(line + band).properties(width=1000, height=400).interactive()

alt.LayerChart(...)

In [ ]:
##Observe também que uma dica de ferramenta foi adicionada ao gráfico; 
##passar o mouse sobre as células revela os valores das variáveis ​​selecionadas
alt.Chart(df_ndvi).mark_rect().encode(
    x='Year:O',
    y='Month:O',
    color=alt.Color(
        'mean(NDVI_mean):Q', scale=alt.Scale(scheme='redyellowgreen', domain=(-1, 1))),
    tooltip=[
        alt.Tooltip('Year:O', title='Year'),
        alt.Tooltip('Month:O', title='Month'),
        alt.Tooltip('mean(NDVI_mean):Q', title='NDVI')
    ]).properties(width=600, height=300)

alt.Chart(...)

# ***Baixando nossa coleção de images***

In [ ]:
##Função para download
download = geemap.ee_export_image_collection_to_drive(l8.select(['NDVI']).limit(3),##Retirar o .limit(3) apenas para exemplificar
                                                      region=region.geometry(), 
                                                      folder='GEE_PYTHON', 
                                                      scale=30,
                                                      crs='EPSG:4674', ##sirgas 2000 epsg:4674
                                                      maxPixels=1e13, 
                                                      fileFormat='GeoTIFF') #Salvará as imagens automaticamente do Drive

Total number of images: 3



Request had insufficient authentication scopes.


In [ ]:
## Agora vamos "zipar" a pasta e realizar o download para o nosso PC
!zip -r /downloadpastaderesultados.zip /content/drive/MyDrive/CURSO_GEE_PYTHON_T2

	zip warning: name not matched: /content/drive/MyDrive/CURSO_GEE_PYTHON_T2

zip error: Nothing to do! (try: zip -r /downloadpastaderesultados.zip . -i /content/drive/MyDrive/CURSO_GEE_PYTHON_T2)


In [ ]:
##Agora realizamos o download da pasta zipada
from google.colab import files
files.download( "/downloadpastaderesultados.zip" ) 